In [1]:
print("OK!")

OK!


In [32]:
# data_management_system_tool.py
import os
import json
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.chains import SequentialChain
from langchain.agents import Tool
from langchain.callbacks import StdOutCallbackHandler
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from dotenv import load_dotenv


ImportError: cannot import name 'Graph' from 'langchain' (c:\Users\Dell\anaconda3\envs\opschatbot\lib\site-packages\langchain\__init__.py)

In [26]:
# Load environment variables
load_dotenv()

# Set OpenAI API Key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


In [28]:
def data_management_system(query_params: dict) -> str:
    # Initialize SparkSession
    spark = SparkSession.builder \
        .appName("DataManagementSystem") \
        .getOrCreate()

    # Connect to Elasticsearch
    es_hosts = ["http://localhost:9200"]
    es = Elasticsearch(es_hosts)

    try:
        # Construct Elasticsearch query based on query_params
        s = Search(using=es, index='logs')
        if 'order_id' in query_params:
            s = s.filter('term', order_id=query_params['order_id'])
        if 'timestamp' in query_params:
            s = s.filter('range', timestamp={'lte': query_params['timestamp']})
        # Add other filters as needed

        # Execute query and get results
        response = s.execute()
        logs = [hit.to_dict() for hit in response]

        # Create DataFrame from logs
        df = spark.createDataFrame(logs)

        # Data Processing using Spark
        # Filtering and Transformation
        df_filtered = df.filter(col('status').isNotNull())
        df_transformed = df_filtered.select('appName', 'status', 'timestamp', 'errorDetails')

        # Correlation (Example: Order by timestamp)
        df_correlated = df_transformed.orderBy('timestamp')

        # Collect results
        processed_logs = df_correlated.collect()

        # Convert processed logs to string
        processed_logs_str = '\n'.join([str(row.asDict()) for row in processed_logs])

        spark.stop()

        return processed_logs_str if processed_logs_str else 'No relevant logs found.'
    except Exception as e:
        spark.stop()
        return f'Error in Data Management System Tool: {str(e)}'


In [29]:
class InterpretQueryNode(Node):
    def run(self, query: str) -> dict:
        # Use LLM to interpret the query and extract parameters
        prompt = f"Extract the intent and parameters from the following query:\n\n{query}\n\nProvide the result in JSON format with 'intent' and 'parameters'."
        interpretation = llm(prompt)
        # Parse the interpretation to get intent and parameters
        try:
            interpretation_json = json.loads(interpretation)
            return {
                "intent": interpretation_json.get("intent"),
                "parameters": interpretation_json.get("parameters")
            }
        except json.JSONDecodeError:
            return {
                "intent": None,
                "parameters": {}
            }


NameError: name 'Node' is not defined